In [ ]:
import mysql.connector

def register_voter():
    conn = None
    cursor = None
    try:
        conn = mysql.connector.connect(host="localhost", user="root", password="root123", database="voting_system")
        cursor = conn.cursor()

        name = input("Enter your name: ")
        email = input("Enter your email: ")

        # Ask for the voter's age
        age = int(input("Enter your age: "))

        # Check if the email already exists
        cursor.execute("SELECT * FROM Voters WHERE voter_email = %s", (email,))
        if cursor.fetchone():
            print("Email already registered!")
            return

        # Check if the voter is 18 or older
        if age < 18:
            print("Sorry, you must be at least 18 years old to register.")
            return

        # Insert the voter into the database with the age provided
        cursor.execute("INSERT INTO Voters (voter_name, voter_email, age) VALUES (%s, %s, %s)", (name, email, age))
        conn.commit()
        print("Registration successful!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

def cast_vote():
    conn = None
    cursor = None
    try:
        conn = mysql.connector.connect(host="localhost", user="root", password="root123", database="voting_system")
        cursor = conn.cursor()

        # Fetch available candidates
        cursor.execute("SELECT * FROM Candidates")
        candidates = cursor.fetchall()
        print("Available Candidates:")
        for candidate in candidates:
            print(f"{candidate[0]}: {candidate[1]}")

        # Get the voter's email and the chosen candidate's ID
        email = input("Enter your email: ")
        candidate_id = int(input("Enter the candidate ID you want to vote for: "))

        # Get voter_id using email
        cursor.execute("SELECT voter_id FROM Voters WHERE voter_email = %s", (email,))
        voter = cursor.fetchone()
        if not voter:
            print("Voter not found. Please register first.")
            return
        voter_id = voter[0]

        # Check if the voter has already voted
        cursor.execute("SELECT * FROM Votes WHERE voter_id = %s", (voter_id,))
        if cursor.fetchone():
            print("You have already voted!")
        else:
            cursor.execute("INSERT INTO Votes (voter_id, candidate_id) VALUES (%s, %s)", (voter_id, candidate_id))
            conn.commit()
            print("Your vote has been cast successfully!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
def display_votes():
    conn = None
    cursor = None
    try:
        conn = mysql.connector.connect(host="localhost", user="root", password="root123", database="voting_system")
        cursor = conn.cursor()
        cursor.execute("""
            SELECT c.candidate_name, COUNT(v.vote_id) AS total_votes
            FROM Candidates c
            LEFT JOIN Votes v ON c.candidate_id = v.candidate_id
            GROUP BY c.candidate_name
        """)
        results = cursor.fetchall()
        print("Total votes for each candidate:")
        for result in results:
            print(f"{result[0]}: {result[1]} votes")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
def main_menu():
    print("Welcome to the Voting System!")
    print("1. Register as a voter")
    print("2. Cast your vote")
    print("3. View total votes for each candidate")
    print("4. Exit")

def start_system():
    while True:
        main_menu()
        choice = input("Please select an option (1-4): ")

        if choice == '1':
            register_voter()
        elif choice == '2':
            cast_vote()
        elif choice == '3':
            display_votes()
        elif choice == '4':
            print("Thank you for using the Voting System!")
            break
        else:
            print("Invalid choice, please try again.")
start_system()

# SQL queries
CREATE DATABASE voting_system;
USE voting_system;

CREATE TABLE Voters (
    voter_id INT AUTO_INCREMENT PRIMARY KEY,
    voter_name VARCHAR(255) NOT NULL,
    voter_email VARCHAR(255) UNIQUE NOT NULL,
    age INT NOT NULL,
    registration_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE Candidates (
    candidate_id INT AUTO_INCREMENT PRIMARY KEY,
    candidate_name VARCHAR(255) NOT NULL
);

CREATE TABLE Votes (
    vote_id INT AUTO_INCREMENT PRIMARY KEY,
    voter_id INT,
    candidate_id INT,
    vote_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (voter_id) REFERENCES Voters(voter_id),
    FOREIGN KEY (candidate_id) REFERENCES Candidates(candidate_id)
);
INSERT INTO Candidates (candidate_name) VALUES ('Alice'), ('Bob'), ('Charlie');
ALTER TABLE Voters ADD COLUMN age INT NOT NULL;

select * from voters;